### Imports

In [1]:
# Libraries
import os
from importlib import reload

import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Scripts
from helpers import data, runners, utils, generators

In [2]:
!pip show tensorflow # Should be v2.10

Name: tensorflow
Version: 2.13.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/antrikshdhand/Documents/github/thesis-ml/ml/virt/lib/python3.9/site-packages
Requires: tensorflow-macos
Required-by: 


In [3]:
!pip show keras # Should be v2.10

Name: keras
Version: 2.13.1
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /Users/antrikshdhand/Documents/github/thesis-ml/ml/virt/lib/python3.9/site-packages
Requires: 
Required-by: tensorflow-macos


In [4]:
reload(runners)
reload(data)
reload(utils)

<module 'helpers.utils' from '/Users/antrikshdhand/Documents/github/thesis-ml/ml/main/helpers/utils.py'>

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Check whether tf is using the GPU
utils.check_gpu_use() 

GPU DETECTED ✓
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## `main`

In [6]:
DATA_BATCH_SIZE = 16
GPU_BATCH_SIZE = 4
NUM_EPOCHS = 20

### Load data

In [9]:
pairs_df = pd.read_csv("../data/DATASET_CSVS/deepship_pairs_diff_recording.csv")

print(pairs_df.shape)

(12880, 5)


In [10]:
# ADD RELATIVE PATH

PATH_TO_ROOT = "..\\data\\"
pairs_df["file_path_1"] = pairs_df["file_path_1"].apply(
    lambda x: PATH_TO_ROOT + x
)
pairs_df["file_path_2"] = pairs_df["file_path_2"].apply(
    lambda x: PATH_TO_ROOT + x
)

pairs_df.head(2)

,ship_name,file_path_1,date_seg_1,file_path_2,date_seg_2
0,ALFRED_N,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170326_1,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170403_51
1,ALFRED_N,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170326_2,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170403_48


In [11]:
# Get testing split
train, test = train_test_split(
    pairs_df,
    test_size=0.2,
    random_state=100,
    shuffle=True
)

In [12]:
# Initialise generators
train_gen = generators.N2NGenerator(
    train,
    'mat',
    'Pexp',
    batch_size=DATA_BATCH_SIZE,
    shuffle=True,
    conv_channel=True
)

test_gen = generators.N2NGenerator(
    test,
    'mat',
    'Pexp',
    batch_size=DATA_BATCH_SIZE,
    shuffle=True,
    conv_channel=True
)

### Initialise model

In [ ]:
def psnr(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

In [14]:
model = keras.saving.load_model('models/saved/n2n_finetuned_25112024/n2n_finetuned.keras')

# from models import unet, unetpro
# reload(unetpro)

# model = unetpro.get_unetpro_model(
#     input_size=(192, 192, 1), 
#     classes=1, # Use 1 for denoising
#     dropout=0.2
# ) 

# # model = unet.get_unet_model(
# #     input_size=(192, 192, 1),
# #     classes=1,
# #     dropout=0.5
# # )

# model.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
#     loss=keras.losses.MeanSquaredError(),
#     metrics=[psnr]
# )

# model.summary()

TypeError: Could not deserialize class 'Functional' because its parent module keras.src.models.functional cannot be imported. Full object config: {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'unet', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 256, 256, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d', 'trainable': False, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'filters': 64, 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'HeNormal', 'config': 

### Train model

In [ ]:
keras.backend.clear_session()

# [DBS=16, GBS=4] Entire dataset + 1 epoch = 24 min
# [DBS=16, GBS=4] Entire dataset + 10 epochs = 235 min

history = model.fit(
    train_gen,
    batch_size=GPU_BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=2
)

In [ ]:
evals = model.evaluate(
    test_gen,
    batch_size=GPU_BATCH_SIZE,
    verbose=2
)

In [ ]:
# model.save('models/saved/unet_denoiser_11112024/unet_denoiser.keras')
model.save('models/saved/unetpro_denoiser_13112024/unetpro_denoiser_10_epochs.keras')

### Evaluation, metrics, and visualising denoised output

In [ ]:
# Get a few samples from the test generator for visualisation and metrics calculation
num_samples = 5 

sample_X, sample_y = next(iter(test_gen))
sample_inputs = sample_X[:num_samples]  # Extract the first `num_samples` examples from the batch
sample_outputs = sample_y[:num_samples]  # Extract the first `num_samples` examples from the batch

# Predict the denoised outputs
denoised_outputs = model.predict(sample_inputs)

In [ ]:
# Get evaluation metrics
mse_score = tf.keras.losses.MeanSquaredError()(sample_inputs, denoised_outputs).numpy()
psnr_score = psnr(sample_inputs, denoised_outputs).numpy()

print(f"MSE on sample batch: {mse_score}")
print(f"PSNR on sample batch: {psnr_score}")

In [ ]:
# Check if all outputs are normalised
for i, output in enumerate(denoised_outputs):
    print(f"Sample {i+1} - Min: {np.min(output)}; Max: {np.max(output)}")

In [ ]:
# Plot the input, output, and target spectrograms for each sample
for i in range(num_samples):
    plt.figure(figsize=(12, 4))
    
    # INPUT
    plt.subplot(1, 3, 1)
    plt.imshow(sample_inputs[i].squeeze(), cmap="viridis", aspect="auto")
    plt.title("Input")
    plt.colorbar()

    # DENOISED OUTPUT
    plt.subplot(1, 3, 2)
    plt.imshow(denoised_outputs[i].squeeze(), cmap="viridis", aspect="auto")
    plt.title("Denoised Output")
    plt.colorbar()
    plt.clim(0, 1)

    # TARGET
    plt.subplot(1, 3, 3)
    plt.imshow(sample_outputs[i].squeeze(), cmap="viridis", aspect="auto")
    plt.title("Target")
    plt.colorbar()
    
    plt.tight_layout()
    plt.show()